In [4]:
import numpy as np
import random
from operator import add

# Define gridworld class
class Gridworld:
    def __init__(self, gridworld_length=2, gridworld_width=10, num_obstacles=10,
                 collisionReward= -1, destinationReward= 10, defaultReward= 0, failChance= 0.1, gamma= 0.9):
        self.gridworld_length = gridworld_length
        self.gridworld_width = gridworld_width
        self.grid = np.zeros((gridworld_length,gridworld_width))
        self.ds_actions = {"u": [0,-1], "d": [0,1], "l": [-1,0], "r": [1,0]}
        self.actions= list(self.ds_actions.keys()),
        self.num_obstacles = num_obstacles
        self.source, self.destination, self.obstacle_positions = self.initiate_gridworld()
        self.agent = self.source
        self.collisionReward = collisionReward
        self.destinationReward = destinationReward
        self.defaultReward = defaultReward
        self.failChance = failChance
        self.gamma = gamma

    def random_coord(self):
        return [random.randint(0, self.gridworld_length-1), random.randint(0, self.gridworld_width-1)]

    def initiate_gridworld(self):
        # add a random source and destination to the gridworld
        source = self.random_coord()
        destination = self.random_coord()
        while destination == source:
            destination = self.random_coord()

        # add some random obstacles to the gridworld, making sure that the source and destination are not obstacles
        obstacle_positions = []
        print(self.num_obstacles)
        while len(obstacle_positions) < self.num_obstacles:
            position = self.random_coord()
            if position != source or position != destination:
                obstacle_positions = obstacle_positions.append(position)
        print(source, destination, obstacle_positions)
        return source, destination, obstacle_positions

    def takeAction(self, a):
        # take action with probability 0.1, stay in same state with probability 0.9
        if random.random() < 1 - self.failChance:
            print(self.agent, self.ds_actions[a])
            new_agent = list(map(add, self.agent, self.ds_actions[a]))
            print(new_agent)
            # if collision
            if new_agent[0] < 0 or new_agent[0] >= self.gridworld_length or \
                new_agent[1] < 0 or new_agent[1] >= self.gridworld_width or \
                new_agent in self.obstacle_positions:
                return self.collisionReward
            self.agent = new_agent
            if new_agent == self.destination:
                return self.destinationReward
        return self.defaultReward

    def print_gridworld(self):
        for row in range(self.gridworld_length):
            for col in range(self.gridworld_width):
                if [row,col] in self.obstacle_positions:
                    print('O', end=' ')
                if [row,col] == self.destination:
                    print('D', end=' ')
                if [row, col] == self.agent:
                    print('A', end=' ')
                else:
                    print('-', end=' ')
            print()

rewards=[]
g = Gridworld()
# simulate the agent's actions
for _ in range(20):
    g.print_gridworld()
    # choose a random action
    action = random.choice(g.actions[0])
    print(f'Taking action: {action}')
    # take the action and update the agent's position
    reward = g.takeAction(action)
    print("Action: ", action, "Reward: ", reward)
    rewards.append(reward)    


10


TypeError: 'NoneType' object is not iterable